# 2b. Create bike network

In [ ]:
# Standard library and path imports
import set_path

# Third-party library imports
import numpy as np
import pandas as pd
import geopandas as gpd
import shapely
import shapely.geometry as sg
import shapely.ops as so
from tqdm.notebook import tqdm_notebook
tqdm_notebook.pandas()
import osmnx as ox
import momepy
import folium

# Local or project-specific imports
import plot_utils
import bgt_utils
import poly_utils
import settings as st

if st.my_run == "azure":
    import config_azure as cf
elif st.my_run == "local":
    import config as cf

## Import BGT data

In [ ]:
# Get BGT data
df_bgt_full = bgt_utils.get_bgt_data_for_bbox(st.bbox, st.bgt_layers + st.bgt_road_layers)
df_bgt_full['naam'].value_counts()

## Select pilot areas

In [ ]:
# Import areas
df_areas = gpd.read_file(cf.output_pilot_area)

In [ ]:
# Only keep BGT data within pilot areas
df_bgt = df_bgt_full.sjoin(df_areas, how='inner', predicate='within') # note: only sidewalk polygons fully inside area are included
df_bgt['naam_left'].value_counts()

## Import bike data from OSM

In [ ]:
# Put buffer around areas to include more lines from OSM
df_areas['buffer'] = df_areas.buffer(100)

# Retrieve the street network from selected area
polygon = df_areas.set_geometry('buffer').to_crs(st.CRS_map).unary_union
cycling_graph = ox.graph_from_polygon(polygon=polygon, network_type='bike', retain_all=True)

In [ ]:
# Fetch nodes and edges of cycling networks
gdf_nodes_cycling, gdf_edges_cycling = ox.graph_to_gdfs(
        cycling_graph,
        nodes=True, edges=True,
        node_geometry=True,
        fill_edge_geometry=True)

## Select which cycling edges to include

In [ ]:
# Select bike polygons
df_bgt_bike_map = df_bgt[df_bgt['naam_left'] == 'fietspad'].to_crs(st.CRS_map).drop('index_right', axis=1)

In [ ]:
# Select correct bike edges
gdf_edges_cycling_sel = gdf_edges_cycling[gdf_edges_cycling['highway'].isin(['cycleway', ['cycleway', 'unclassified'],  ['unclassified', 'cycleway'], 
                                                                           ['cycleway', 'service'], ['residential', 'cycleway']])]
gdf_edges_cycling_sel.shape

In [ ]:
# Buffer BGT bike polygons
df_bgt_bike_map_buffer = df_bgt_bike_map.copy()
df_bgt_bike_map_buffer['geometry'] = df_bgt_bike_map_buffer.buffer(0.0001, single_sided=True)

# Select edges within buffered polygons
gdf_edges_cycling_sel['unique_id'] = range(0, len(gdf_edges_cycling_sel))
gdf_edges_cycling_is = gdf_edges_cycling_sel.sjoin(df_bgt_bike_map_buffer, predicate='intersects', how='inner')
gdf_edges_cycling_is = gdf_edges_cycling_is.drop_duplicates('unique_id')  # drop duplicates/in multiple polygons
gdf_edges_cycling_is.shape

## Add crossing column

In [ ]:
# Select road polygons
road_names = ['rijbaan lokale weg', 'rijbaan regionale weg', 'rijbaan autoweg', 'rijbaan autosnelweg', 'OV-baan']
df_bgt_road_map = df_bgt[df_bgt['naam_left'].isin(road_names)].to_crs(st.CRS_map).drop('index_right', axis=1)

# Create multipolygon of all roads
road_mpg = gpd.GeoDataFrame(geometry=gpd.GeoSeries(df_bgt_road_map['geometry'].unary_union))['geometry'].iloc[0]

In [ ]:
# Cut bike network lines by road polygons
gdf_edges_cycling_is['geometry_split'] = gdf_edges_cycling_is['geometry'].apply(lambda x: so.split(x, road_mpg)) 

# Explode cutted bike linestrings
gdf_edges_cycling_is_ex = gpd.GeoDataFrame(gdf_edges_cycling_is['geometry_split'].explode(index_parts=True), geometry='geometry_split', crs=st.CRS_map)
gdf_edges_cycling_is_ex.rename(columns={'geometry_split': 'geometry'}, inplace=True)
gdf_edges_cycling_is_ex = gdf_edges_cycling_is_ex.set_geometry('geometry')

# Check if (cutted) lines are within the road multipolygon
road_mpg_buffered = road_mpg.buffer(1e-10)
gdf_edges_cycling_is_ex['within_polygon'] = gdf_edges_cycling_is_ex.within(road_mpg_buffered)

In [ ]:
# Create crossing columns
gdf_edges_cycling_is_ex['crossing'] = np.where(gdf_edges_cycling_is_ex['within_polygon'] == True, 'Yes', 'No')
gdf_edges_cycling_is_ex['crossing_type'] = np.where(gdf_edges_cycling_is_ex['crossing'] == 'Yes', 'osm_bike', np.nan)

## Create missing crossings

In [ ]:
# Get all nodes
G = momepy.gdf_to_nx(gdf_edges_cycling_is_ex.to_crs(st.CRS), approach="primal", multigraph=False)
all_nodes = momepy.nx_to_gdf(G, points=True, lines=False)

# Get end nodes
G1 = G.copy()
G1.remove_nodes_from((n for n,d in G.degree() if d!=1))
end_nodes = momepy.nx_to_gdf(G1, points=True, lines=False)

In [ ]:
# Select bike polygons
df_bgt_bike = df_bgt[df_bgt['naam_left'] == 'fietspad']

# Merge bike polygons
df_polygons = gpd.GeoDataFrame(geometry=gpd.GeoSeries(df_bgt_bike['geometry'].unary_union))
df_polygons = gpd.GeoDataFrame(df_polygons.geometry.explode(index_parts=True)).set_crs(st.CRS)

# Ignore polygons that are too small
df_polygons['area'] = df_polygons['geometry'].area
df_polygons = df_polygons[df_polygons.area > st.min_area_size].reset_index()

# Ignore polygons without nodes
polygons_with_nodes = df_polygons.sjoin(all_nodes)['level_1'].unique()
df_polygons = df_polygons[df_polygons['level_1'].isin(polygons_with_nodes)] 

In [ ]:
H = G.copy()

for i in range(len(end_nodes)):
    #print(i)
    # Get location single new node (of curb ramp)
    end_node_loc = end_nodes['geometry'].values[i] 
    
    # Find nearest sidewalk (max n meter away)
    my_distances = df_polygons['geometry'].distance(end_node_loc)
    my_distances = my_distances[my_distances > 3]
    smallest_distance = my_distances.sort_values().iloc[[0]]

    if smallest_distance.iloc[0] < 16:
        df_sidewalk_nb = df_polygons[df_polygons.index.isin(smallest_distance.index)][['geometry']]
        nearby_polygon = df_sidewalk_nb['geometry'].values[0]

        # Get nodes on nearest bike path polygon
        df_nearby_polygon_nodes = all_nodes[all_nodes.within(nearby_polygon)]
        nearby_polygon_nodes = sg.MultiPoint(df_nearby_polygon_nodes['geometry'].values)

        # Find nearest existing node on nearest bike path polygon
        nearby_points = so.nearest_points(end_node_loc, nearby_polygon_nodes)
        end_node = (nearby_points[0].x, nearby_points[0].y)
        connect_node = (nearby_points[1].x, nearby_points[1].y)

        # Add edge between new and existing node
        node_dist = nearby_points[0].distance(nearby_points[1])
        if node_dist < 20:
            H.add_edge(end_node, connect_node, geometry=sg.LineString([end_node, connect_node]), new='Yes')

# Create dataframe with existing and new edges/lines
df_H = momepy.nx_to_gdf(H, points=False, lines=True)
df_H_new = df_H[df_H['crossing'].isna()]

In [ ]:
# Reverse lines, to have both directions
df_H_new_reverse = gpd.GeoDataFrame(df_H_new['geometry'].apply(poly_utils.reverse_line))
df_H_new = pd.concat([df_H_new, df_H_new_reverse])

In [ ]:
# Fill in crossings columns of new lines
df_H_new['crossing'] = 'Yes'
df_H_new['crossing_type'] = 'created_bike'

# Add new lines to existing dataframe
gdf_edges_cycling_is_ex = pd.concat([gdf_edges_cycling_is_ex, df_H_new.to_crs(st.CRS_map)])

## Visualize bike network

In [ ]:
# create buffer for validation direction (left-handed)
gdf_edges_cycling_is['buffer'] = gdf_edges_cycling_is.buffer(0.0001, single_sided=True)

In [ ]:
# set True for satelite background, False for standard background
satellite = False

# Create Folium map
map = folium.Map(
    location=[52.350547922223434, 4.794019242371844], tiles=plot_utils.generate_map_params(satellite=satellite),
    min_zoom=10, max_zoom=25, zoom_start=17,
    zoom_control=True, control_scale=True, control=False
    )

# Add bike paths
folium.GeoJson(df_bgt[df_bgt['naam_left'] == 'fietspad'], style_function=lambda x: {"fillColor": "red"}).add_to(map)

# Add sidewalks  
folium.GeoJson(df_bgt[df_bgt['naam_left'].isin(['voetpad', 'inrit', 'voetgangersgebied'])], style_function=lambda x: {"fillColor": "blue"}).add_to(map)

# Add roads
folium.GeoJson(df_bgt_road_map, style_function=lambda x: {"fillColor": "grey"}).add_to(map)

# Add all cycling edges
folium.GeoJson(gdf_edges_cycling['geometry'], style_function=lambda x: {"color": 'pink', "weight": 1}).add_to(map)

# Add buffer of selected cycling edges (to validate directions)
folium.GeoJson(gdf_edges_cycling_is['buffer'], style_function=lambda x: {"color": 'orange', "weight": 1}).add_to(map)

# Add selected cycling edges
folium.GeoJson(gdf_edges_cycling_is_ex['geometry'], style_function=lambda x: {"color": 'red', "weight": 2}).add_to(map)

# Add cycling crossings
folium.GeoJson(gdf_edges_cycling_is_ex[gdf_edges_cycling_is_ex['crossing_type']=='created_bike'], style_function=lambda x: {"color": 'purple', "weight": 4}).add_to(map)
folium.GeoJson(gdf_edges_cycling_is_ex[gdf_edges_cycling_is_ex['crossing_type']=='osm_bike'], style_function=lambda x: {"color": 'darkred', "weight": 4}).add_to(map)

map

In [ ]:
# Store map
map.save(cf.bike_network_map)

## Store

In [ ]:
# Prepare final bike network dataframe
gdf_final = gdf_edges_cycling_is_ex.reset_index(drop=True)
gdf_final['bikepath_id'] = range(0, len(gdf_final))
gdf_final = gdf_final.to_crs(st.CRS)
gdf_final['length'] = gdf_final.length
gdf_final = gdf_final[['bikepath_id', 'geometry', 'length', 'crossing', 'crossing_type']]
gdf_final.tail(3)

In [ ]:
# Write bike network to file
gdf_final.reset_index().to_file(cf.output_bike_network, driver='GPKG')

In [ ]:
# Prepare bike path polygon dataframe
gdf_bikepaths = df_bgt_bike_map[['geometry']].reset_index(drop=True)
gdf_bikepaths = gdf_bikepaths.to_crs(st.CRS)
gdf_bikepaths.head(3)

In [ ]:
# Write bike path polygons related to bike network to file
gdf_bikepaths.to_file(cf.output_bikepaths_bike_network, driver='GPKG')